<a href="https://colab.research.google.com/github/aoyerinde/python-learning/blob/master/RDD_IBM_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-2.3.1/spark-2.3.1-bin-hadoop2.7.tgz
!tar xf spark-2.3.1-bin-hadoop2.7.tgz
!pip install -q findspark

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [564 B]
Get:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release [564 B]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [819 B]
Get:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release.gpg [833 B]
Get:8 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:10 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ Packages

In [7]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.3.1-bin-hadoop2.7"

import findspark
findspark.init()
from pyspark import SparkContext
sc = SparkContext.getOrCreate()

import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate() 
spark

In [9]:
sc.version

'2.3.1'

In [0]:
x = (1,2,3,4,5,6,7,8,9,10)
x_nbr_rdd = sc.parallelize(x)


In [21]:
x_nbr_rdd.first()
x_nbr_rdd.take(5)

[1, 2, 3, 4, 5]

In [22]:
y = ["Hello", "my Name is Spark"]
y_str_rdd = sc.parallelize(y)
y_str_rdd.take(1)

['Hello']

In [26]:
x_nbr_rdd = x_nbr_rdd.map(lambda x: x+1)
x_nbr_rdd.collect()

[4, 5, 6, 7, 8, 9, 10, 11, 12, 13]

In [29]:
x2 = ["1,2,3,4,5,6,7,8,9,10"]
y_rd = sc.parallelize(x2)
y_rd

ParallelCollectionRDD[13] at parallelize at PythonRDD.scala:184

In [0]:
sum_rd = y_rd.map(lambda y:y.split(",")).map(lambda y:(int (y[2])+int(y[9])))

In [31]:
sum_rd.first()

13

In [33]:
words = ["Hello Human I am Spark and I love running analysis"]
words_rd = sc.parallelize(words)
words_rd.first()

'Hello Human I am Spark and I love running analysis'

In [34]:
words_rd2 = words_rd.map(lambda line: line.split(" "))
words_rd2.first()

['Hello',
 'Human',
 'I',
 'am',
 'Spark',
 'and',
 'I',
 'love',
 'running',
 'analysis']

In [39]:
words_rd2.count()
words_rd2 = words_rd.flatMap(lambda line: line.split(" "))
words_rd2.take(3)

['Hello', 'Human', 'I']

#Pair RDD

In [40]:
z = ["First, Line", "Second,Line", "and,third,Line"]#
z_str_rdd = sc.parallelize(z)
z_str_rdd.first()

'First, Line'

In [42]:
z_rdd_split = z_str_rdd.flatMap(lambda line: line.split(","))
z_rdd_split.collect()

['First', ' Line', 'Second', 'Line', 'and', 'third', 'Line']

In [43]:
countwords = z_rdd_split.map(lambda defined: (defined, 1))
countwords.collect()

[('First', 1),
 (' Line', 1),
 ('Second', 1),
 ('Line', 1),
 ('and', 1),
 ('third', 1),
 ('Line', 1)]

In [44]:
from operator import add
countwords2 = countwords.reduceByKey(add)
countwords2.collect()

[('Second', 1),
 ('Line', 2),
 ('third', 1),
 ('First', 1),
 (' Line', 1),
 ('and', 1)]

In [48]:
#word_rd3 = z_rdd_split.filter(lambda line: "Second" in line)
word_rd3.collect()

['Second']

In [49]:
!rm Readme.md* -f
!wget https://raw.githubusercontent.com/carloapp2/SparkPOT/master/README.md

--2019-09-23 21:32:24--  https://raw.githubusercontent.com/carloapp2/SparkPOT/master/README.md
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3624 (3.5K) [text/plain]
Saving to: ‘README.md’

README.md           100%[===================>]   3.54K  --.-KB/s    in 0s      

2019-09-23 21:32:24 (89.7 MB/s) - ‘README.md’ saved [3624/3624]



In [51]:
textfile_rdd = sc.textFile("README.md")
textfile_rdd.count()

98

In [54]:
textfile_rdd_filter =  textfile_rdd.filter(lambda line: "Spark" in line)
textfile_rdd_filter.first()
textfile_rdd_filter.count()

19

In [60]:
temp =  textfile_rdd_filter.flatMap(lambda line: line.split(" ")).map(lambda\
             x: (x, 1)).reduceByKey(add)
temp.filter(lambda x: x[0].startswith("Spark")).collect()

[('Spark', 14),
 ('Spark"](http://spark.apache.org/docs/latest/building-spark.html).', 1),
 ('SparkPi', 2),
 ('Spark](#building-spark).', 1),
 ('Spark.', 1)]

In [61]:
temp.filter(lambda x: "Spark" in x[0]).collect()

[('Spark', 14),
 ('Spark"](http://spark.apache.org/docs/latest/building-spark.html).', 1),
 ('SparkPi', 2),
 ('Spark](#building-spark).', 1),
 ('Spark.', 1),
 ('tests](https://cwiki.apache.org/confluence/display/SPARK/Contributing+to+Spark#ContributingtoSpark-AutomatedTesting).',
  1)]

In [62]:
!rm Scores.txt* -f
!wget https://raw.githubusercontent.com/carloapp2/SparkPOT/master/Scores.txt

--2019-09-23 22:46:22--  https://raw.githubusercontent.com/carloapp2/SparkPOT/master/Scores.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 75 [text/plain]
Saving to: ‘Scores.txt’

Scores.txt          100%[===================>]      75  --.-KB/s    in 0s      

2019-09-23 22:46:22 (12.9 MB/s) - ‘Scores.txt’ saved [75/75]



In [65]:
raw_rdd = sc.textFile("Scores.txt")
sumscores = raw_rdd.map(lambda l: l.split(",")).map(lambda v: (v[0], int(v[1])\
                 +int(v[2])+int(v[2])+int(v[3])))
final = sumscores.map(lambda avg: (avg[0], avg[1], avg[1]/4.0))

final.take(5)

[('Carlo', 14, 3.5),
 ('Mokhtar', 17, 4.25),
 ('Jacques', 12, 3.0),
 ('Braden', 13, 3.25),
 ('Chris', 18, 4.5)]